In [1]:
import os 
import cv2
import matplotlib.pyplot as plt
import numpy as np

def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

class Convert_to_line:
    def __init__(self,im):
        self.im=im
    
    def get_concat_h(self,im1, im2):
        dst = Image.new('RGB', (im1.width + im2.width, im1.height))
        dst.paste(im1, (0, 0))
        dst.paste(im2, (im1.width, 0))
        return dst
    

    #Rightside crop    
    def right_crop(self):
        #Top Crop
        width,height=self.im.size
        top=0.15*height
        bottom=1.015*height/2

        right=0.9*width
        left=1*width/2
        im1=self.im.crop((left,top,right,bottom))


        #Bottom Crop
        top=1.015*height/2
        bottom=0.875*height

        right=0.9*width
        left=1*width/2
        im2=self.im.crop((left,top,right,bottom)).resize(im1.size)

        IM1=get_concat_h(im2, im1)
        return IM1

    def left_crop(self):
        #Leftside crop
        #TopCrop
        width,height=self.im.size
        top=0.15*height
        bottom=1.015*height/2

        left=1*width/9
        right=1*width/2.1
        im1=self.im.crop((left,top,right,bottom))

        #Bottom Crop
        top=1.015*height/2
        bottom=0.875*height

        left=1*width/9
        right=1*width/2.1
        im2=self.im.crop((left,top,right,bottom)).resize(im1.size)

        imlist =[im1,im2]
        IM2=get_concat_h(im2, im1)
        return IM2
    
    def merge(self,IM2, IM1):
        IM=get_concat_h(IM1, IM2)
        return IM

In [ ]:
#Step 1: Convert pdf to png
import glob, sys, fitz

# To get better resolution
zoom_x = 2.0  # horizontal zoom
zoom_y = 2.0  # vertical zoom
mat = fitz.Matrix(zoom_x, zoom_y)  # zoom factor 2 in each dimension

all_files = glob.glob(r"C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Raw_Data/1940/All.pdf")

for filename in all_files:
    doc = fitz.open(filename)  # open document
    for page in doc:  # iterate through the pages
        if len(str(page.number))==1:
            print("")
        if len(str(page.number))==2:
            pix = page.get_pixmap(matrix=mat)  # render page to an image
            pix.save(r"C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Raw_Data/1940/Lines0/0%i-page.png" % page.number)  # store image as a PNG
        else:
            pix = page.get_pixmap(matrix=mat)  # render page to an image
            pix.save(r"C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Raw_Data/1940/Lines0/%i-page.png" % page.number)  # store image as a PNG

In [41]:
#Step2
#Convert to line
from PIL import Image
import os, cv2

def listdir_nohidden1(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f
            
pathname0 = r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Raw_Data/1940/Lines0'
pathname = r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Raw_Data/1940/Lines'
for i in range(0,len(sorted(listdir_nohidden1(pathname0)))):
    image = sorted(listdir_nohidden1(pathname0))[i]
    image_test = Image.open(pathname0+"/"+image)
    
    test=Convert_to_line(image_test)
    line=test.merge(test.right_crop(),test.left_crop())
    
    line.save(pathname+"/"+image)


In [78]:
#Combine pages to single line
from PIL import Image

pathname = r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Raw_Data/1940/Lines'

allfiles = os.listdir(pathname)
imlist =[cv2.imread(pathname+"/"+filename) for filename in np.flip(allfiles) if filename[-4:] in [".png"]]
im = np.hstack(imlist)


In [79]:
#Detecting circles
cropped_image = im[0:45, 0:im.shape[1]]

# Detecting circles: This step is heavily refering to the following website https://www.geeksforgeeks.org/circle-detection-using-opencv-python/
# Convert to grayscale.
gray = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
# Blur using 3 * 3 kernel.
gray_blurred = cv2.blur(gray, (3, 3))  

# Apply Hough transform on the blurred image.
detected_circles = cv2.HoughCircles(gray_blurred, 
                   cv2.HOUGH_GRADIENT, 1, 150, param1 = 50,
               param2 = 15, minRadius = 8, maxRadius = 12)
detected_circles = np.uint16(np.around(detected_circles))

#The Detection can be eye-checked by projecting an annotated image
#Skip this step for automation.
#if detected_circles is not None:
    # Convert the circle parameters a, b and r to integers.
#for pt in detected_circles[0, :]:
#    a, b, r = pt[0], pt[1], pt[2]
#
#    # Draw the circumference of the circle.
#    cv2.circle(cropped_image, (a, b), r, (0, 255, 0), 2)
#
#    # Draw a small circle (of radius 1) to show the center.
#    cv2.circle(cropped_image, (a, b), 1, (0, 0, 255), 3)
#
#    cv2.imshow("Detected Circle", cropped_image)
#    cv2.waitKey(0)


In [80]:
#Crop into departments
detected_circles = np.uint16(np.around(detected_circles))
circle_list=sorted(detected_circles[0], key=lambda x: x[0])

pathname = r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Raw_Data/1940/Departments'
for i in range(0,len(detected_circles[0])):
    if i ==0:
        dept = im[0:im.shape[0], 0:circle_list[0][0]+3]
        Image.fromarray(dept).save(pathname+"/Dept00"+str(i)+".png")
    if i>0 and len(str(i))==1:
        dept = im[0:im.shape[0], circle_list[i-1][0]+3:circle_list[i][0]+3]
        Image.fromarray(dept).save(pathname+"/Dept00"+str(i)+".png")
    if i>0 and len(str(i))==2:
        dept = im[0:im.shape[0], circle_list[i-1][0]+3:circle_list[i][0]+3]
        Image.fromarray(dept).save(pathname+"/Dept0"+str(i)+".png")
    else:
        dept = im[0:im.shape[0], circle_list[i-1][0]+3:circle_list[i][0]+3]
        Image.fromarray(dept).save(pathname+"/Dept"+str(i)+".png")

In [21]:
cv2.imshow('Horizontal Appended', concat_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
import os
from os import listdir

import cv2
import matplotlib.pyplot as plt
import numpy as np

for images in os.listdir(): 
    if (images.endswith(".jpg")):
        im=cv2.imread(images,cv2.IMREAD_COLOR)
        
        # Detecting circles: This step is heavily refering to the following website https://www.geeksforgeeks.org/circle-detection-using-opencv-python/
        # Convert to grayscale.
        gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        # Blur using 3 * 3 kernel.
        gray_blurred = cv2.blur(gray, (3, 3))  
        # Apply Hough transform on the blurred image.
        detected_circles = cv2.HoughCircles(gray_blurred, 
                           cv2.HOUGH_GRADIENT, 1, 100, param1 = 50,
                       param2 = 15, minRadius = 5, maxRadius = 10)
        #The Detection can be eye-checked by projecting an annotated image
        #Skip this step for automation.
        if detected_circles is not None:
            # Convert the circle parameters a, b and r to integers.
            detected_circles = np.uint16(np.around(detected_circles))

            for pt in detected_circles[0, :]:
                a, b, r = pt[0], pt[1], pt[2]

                # Draw the circumference of the circle.
                cv2.circle(im, (a, b), r, (0, 255, 0), 2)

                # Draw a small circle (of radius 1) to show the center.
                cv2.circle(im, (a, b), 1, (0, 0, 255), 3)

                cv2.imshow("Detected Circle", im)
                cv2.waitKey(0)